# Dependencies

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Load images
   - Link to the original image: [pexels.com/photo/areal-view-of-lake-bridge-and-trees-during-daytime-145525](https://www.pexels.com/photo/areal-view-of-lake-bridge-and-trees-during-daytime-145525/)

In [ ]:
# note: open-cv loads images in BGR format.
scene_1 = cv2.imread('../assets/images/third_party/nature_1.jpg')
scene_2 = cv2.imread('../assets/images/third_party/nature_2.jpg')

# plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))
axs[0].imshow(cv2.cvtColor(scene_1, code=cv2.COLOR_BGR2RGB))
axs[0].set_title("Scene 1")
axs[0].axis('off')
axs[1].imshow(cv2.cvtColor(scene_2, code=cv2.COLOR_BGR2RGB))
axs[1].set_title("Scene 2")
axs[1].axis('off')
plt.show()

# Image Stitching
   - It is the process of combining multiple images to create a single larger image, often referred to as a panorama.

## 1. Key-Point detection using Scale-Invariant Feature Transform [SIFT]
   - [SIFT](https://docs.opencv.org/4.x/da/df5/tutorial_py_sift_intro.html) is a corner detector like [Harris](https://docs.opencv.org/4.x/dc/d0d/tutorial_py_features_harris.html) (a key-point detector since corners are good candidates for key-points)

In [3]:
# BGR to GrayScale
scene_1_gray = cv2.cvtColor(scene_1, cv2.COLOR_BGR2GRAY)
scene_2_gray = cv2.cvtColor(scene_2, cv2.COLOR_BGR2GRAY)

In [4]:
# create a SIFT object
sift = cv2.SIFT_create()

# detect keypoints and compute descriptors for the images
keypoints_1, descriptors_1 = sift.detectAndCompute(scene_1, None)
keypoints_2, descriptors_2 = sift.detectAndCompute(scene_2, None)

In [ ]:
# create a new image including key-points
scene_1_kp = cv2.drawKeypoints(scene_1, keypoints_1, None)
scene_2_kp = cv2.drawKeypoints(scene_2, keypoints_2, None)

# plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))
axs[0].imshow(cv2.cvtColor(scene_1_kp, cv2.COLOR_BGR2RGB))
axs[0].set_title(f"Reference [#keypoints:{descriptors_1.shape[0]} - #features per keypoint:{descriptors_1.shape[1]}]")
axs[0].axis('off')
axs[1].imshow(cv2.cvtColor(scene_2_kp, cv2.COLOR_BGR2RGB))
axs[1].set_title(f"Target [#keypoints:{descriptors_2.shape[0]} - #features per keypoint:{descriptors_2.shape[1]}]")
axs[1].axis('off')
plt.show()

## 2. Feature/Key-Point Matching with FLANN
   - [cv2.FlannBasedMatcher](https://docs.opencv.org/4.x/d5/d6f/tutorial_feature_flann_matcher.html)
   - [cv2.FlannBasedMatcher.knnMatch](https://docs.opencv.org/4.x/db/d39/classcv_1_1DescriptorMatcher.html#a378f35c9b1a5dfa4022839a45cdf0e89)

In [6]:
# match the keypoints between the two images
matcher = cv2.FlannBasedMatcher()
matches = matcher.knnMatch(descriptors_1, descriptors_2, k=2)

## 3. Apply thresholding

In [7]:
# apply ratio test to get good matches [thresholding]
good_matches = []
for m, n in matches:
    if m.distance < 0.4 * n.distance:
        good_matches.append(m)

# create a new image showing the good matches
matches_image = cv2.drawMatches(scene_1, keypoints_1, scene_2, keypoints_2, good_matches, None)

In [ ]:
# plot
plt.figure(figsize=(16, 8))
plt.imshow(cv2.cvtColor(matches_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title(f"Number of matched key-points: {len(good_matches)}")

## 4. Homography
   - [cv2.findHomography](https://docs.opencv.org/4.x/d7/dff/tutorial_feature_homography.html) is used to estimate a perspective transformation (homography) between two sets of corresponding points in different images.
   - [cv2.perspectiveTransform](https://docs.opencv.org/4.x/d2/de8/group__core__array.html#gad327659ac03e5fd6894b90025e6900a7) performs the perspective matrix transformation of vectors.
   - [cv2.warpPerspective](https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87) applies a perspective transformation to an image.

In [10]:
src_pts = np.float32([keypoints_1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints_2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# find the homography matrix
transformation_matrix, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, ransacReprojThreshold=10)

# log
print(f"Transformation:\n{transformation_matrix}")

In [23]:
# merging two images based on best matched key-points
height1, width1 = scene_1.shape[:2]
height2, width2 = scene_2.shape[:2]

corners1 = np.float32([[0, 0], [0, height1], [width1, height1], [width1, 0]]).reshape(-1, 1, 2)
corners2 = np.float32([[0, 0], [0, height2], [width2, height2], [width2, 0]]).reshape(-1, 1, 2)

corners2_transformed = cv2.perspectiveTransform(corners2, transformation_matrix)
all_corners = np.concatenate((corners1, corners2_transformed), axis=0)

[x_min, y_min] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
[x_max, y_max] = np.int32(all_corners.max(axis=0).ravel() + 0.5)

translation_matrix = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]])

panorama = cv2.warpPerspective(scene_1, translation_matrix.dot(transformation_matrix), (x_max - x_min, y_max - y_min))
panorama[-y_min:height1 - y_min, -x_min:width1 - x_min] = scene_2

In [ ]:
# plot
fig = plt.figure(figsize=(8, 8), constrained_layout=True)
gs = fig.add_gridspec(2, 2)
ax1 = fig.add_subplot(gs[0, 0])
ax1.imshow(cv2.cvtColor(scene_1, cv2.COLOR_BGR2RGB))
ax1.set_title('Scene 1')
ax1.axis('off')
ax2 = fig.add_subplot(gs[0, 1])
ax2.imshow(cv2.cvtColor(scene_2, cv2.COLOR_BGR2RGB))
ax2.set_title('Scene 2')
ax2.axis('off')
ax3 = fig.add_subplot(gs[1, :])
ax3.imshow(cv2.cvtColor(panorama, cv2.COLOR_BGR2RGB))
ax3.set_title('Merged')
ax3.axis('off')
plt.show()